In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go

In [ ]:
def generate_daily_visitors(mean, sd, days):
    return np.random.normal(mean, sd, days).astype(int)

def generate_daily_purchasers(mean, sd, days):
    return np.random.normal(mean, sd, days).astype(int)

def calculate_z_score(p_control, p_variant, n_control, n_variant):
    p_pool = (p_control * n_control + p_variant * n_variant) / (n_control + n_variant)
    se = np.sqrt(p_pool * (1 - p_pool) * (1/n_control + 1/n_variant))
    return (p_variant - p_control) / se


def calculate_p_value(z_score):
    return norm.sf(abs(z_score)) * 2  

def simulate_aa_test(num_pairs, mean_visitors, sd_visitors, mean_purchasers, sd_purchasers, num_days):
    results = []
    p_values_list = []
    
    for instance in range(1, num_pairs + 1):
        visitors_control = generate_daily_visitors(mean_visitors, sd_visitors, num_days)
        visitors_variant = visitors_control  
        purchasers_control = generate_daily_purchasers(mean_purchasers, sd_purchasers, num_days)
        purchasers_variant = generate_daily_purchasers(mean_purchasers, sd_purchasers, num_days)
        
        cum_visitors_control, cum_visitors_variant = 0, 0
        cum_purchasers_control, cum_purchasers_variant = 0, 0
        false_positive_duration = False
        false_positive_weekly = False
        false_positive_continuous = False
        
        for day in range(1, num_days + 1):
            cum_visitors_control += visitors_control[day - 1]
            cum_visitors_variant += visitors_variant[day - 1]
            cum_purchasers_control += purchasers_control[day - 1]
            cum_purchasers_variant += purchasers_variant[day - 1]
            
            p_control = cum_purchasers_control / cum_visitors_control
            p_variant = cum_purchasers_variant / cum_visitors_variant
            
            z_score = calculate_z_score(p_control, p_variant, cum_visitors_control, cum_visitors_variant)
            p_value = calculate_p_value(z_score)
            
            p_values_list.append({'Day': day, 'Instance': instance, 'P_Value': p_value})
            
            if day == num_days and p_value < 0.05:
                false_positive_duration = True
            if day % 7 == 0 and p_value < 0.05:
                false_positive_weekly = True
            if p_value < 0.05:
                false_positive_continuous = True
        
        results.append({
            'Instance': instance,
            'False_Positive_Duration': false_positive_duration,
            'False_Positive_Weekly': false_positive_weekly,
            'False_Positive_Continuous': false_positive_continuous
        })
    
    return pd.DataFrame(results), pd.DataFrame(p_values_list)


In [8]:
# Parameters
num_pairs = 1000  
mean_visitors = 1000000
sd_visitors = 0
num_days = 28
mean_purchasers = 120000
sd_purchasers = 324.96


df_results, df_p_values = simulate_aa_test(num_pairs, mean_visitors, sd_visitors, mean_purchasers, sd_purchasers, num_days)



In [10]:
def plot_false_positive_comparison(df):
    false_positive_rates = {
        "Duration": df['False_Positive_Duration'].mean(),
        "Weekly Peeking": df['False_Positive_Weekly'].mean(),
        "Continuous Peeking": df['False_Positive_Continuous'].mean()
    }
    
    fig = go.Figure()
    for key, value in false_positive_rates.items():
        fig.add_trace(go.Bar(x=[key], y=[value], text=[f"{value:.2%}"], textposition='auto'))
    
    fig.update_layout(title="False Positive Rates Under Different Peeking Strategies", 
                      yaxis=dict(tickformat=".2%"),
                      xaxis_title="Peeking Strategy",
                      yaxis_title="False Positive Rate")
    fig.show()

plot_false_positive_comparison(df_results)

In [12]:
def plot_selected_p_values(df_p_values, num_samples=10):
    selected_instances = np.random.choice(df_p_values['Instance'].unique(), num_samples, replace=False)
    
    fig = go.Figure()
    for instance in selected_instances:
        instance_data = df_p_values[df_p_values['Instance'] == instance]
        fig.add_trace(go.Scatter(x=instance_data['Day'], 
                                 y=instance_data['P_Value'], 
                                 mode='lines', 
                                 name=f'Instance {instance}'))
    
    fig.add_trace(go.Scatter(x=[1, df_p_values['Day'].max()], y=[0.05, 0.05], mode='lines', 
                             name='Significance Threshold', line=dict(dash='dash', color='red')))
    
    fig.update_layout(title="P-Value Progression for Selected AA Test Instances", 
                      xaxis_title="Day", yaxis_title="P-Value")
    fig.show()

plot_selected_p_values(df_p_values)


In [11]:
df_results.head()

,Instance,False_Positive_Duration,False_Positive_Weekly,False_Positive_Continuous
0,1,False,False,True
1,2,False,False,False
2,3,False,False,True
3,4,False,False,False
4,5,False,False,False
